In [1]:
#!/usr/bin/env python
import glafic
from tqdm import tqdm
import numpy as np
import psutil
import shutil
import os
import time
import requests
import json  # ### NEW ### - For handling the restart file
import pandas as pd
import re

In [2]:
# Define the lens corresponding parameters (order preserved)
# POW
pow_params = ['$z_{s,fid}$', 'x', 'y', 'e', '$θ_{e}$', '$r_{Ein}$', '$\gamma$ (PWI)']

# SIE
sie_params = ['$\sigma$', 'x', 'y', 'e', '$θ_{e}$', '$r_{core}$', 'NaN']

# NFW
nfw_params = ['M', 'x', 'y', 'e', '$θ_{e}$', 'c or $r_{s}$', 'NaN']

# EIN
ein_params = ['M', 'x', 'y', 'e', '$θ_{e}$', 'c or $r_{s}$', r'$\alpha_{e}$']

# SHEAR 
shear_params = ['$z_{s,fid}$', 'x', 'y', '$\gamma$', '$θ_{\gamma}$', 'NaN', '$\kappa$']

# Sersic
sersic_params = ['$M_{tot}$', 'x', 'y', 'e', '$θ_{e}$', '$r_{e}$', '$n$']

# Cored SIE
cored_sie_params = ['M', 'x', 'y', 'e', '$θ_{e}$', '$r_{core}$', 'NaN']

# Multipoles
mpole_params = ['$z_{s,fid}$', 'x', 'y', '$\epsilon$', '$θ_{m}$', 'm', 'n']

model_list = ['POW', 'SIE', 'ANFW', 'EIN', 'PERT', 'SERS', 'MPOLE']
model_params = {
    'POW': pow_params,
    'SIE': sie_params,
    'ANFW': nfw_params,
    'EIN': ein_params,
    'PERT': shear_params,
    'SERS': sersic_params,
    'MPOLE' : mpole_params
}

In [ ]:
from itertools import product

base_path = '/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/'
sim_version = 'Sim 7'

m = [round(x, 3) for x in np.linspace(0.01, 0.5, 50)]
n = [round(x, 1) for x in np.linspace(0, 360, 10)]
o = [round(x, 2) for x in np.linspace(-0.5, 0.5, 10)]

chunk_size = 1000

num_files = (len(m) * len(n) * len(o)) // chunk_size

# Read all files 
files = [f"{base_path}{sim_version}_summary_{i+1}.csv" for i in range(num_files)]
print(f"Reading {len(files)} files...")

model_name = 'POW'
macro_model_params = model_name.strip().split('_')[0]
macro_columns = model_params[macro_model_params]

columns=['strength', 'pa', 'kappa', 'num_images', 'pos_rms', 'mag_rms', 't_shear_str', 't_shear_pa', 't_shear_kappa', 'chi2'] + macro_columns

# Read individual files and check if all models are present 
for file in files:
    try:
        df = pd.read_csv(file, names=columns)
        if len(df) - 1 != chunk_size:
            print(f"Warning: {file} has {len(df) - 1} rows, expected {chunk_size}.")
            # Create tuples of (strength, pa, kappa) from current file
            current_models = set(zip(df['strength'].astype(str), df['pa'].astype(str), df['kappa'].astype(str)))
            # Create expected models for current chunk based on indices
            current_idx = int(file.split('_')[-1].split('.')[0]) - 1
            start_idx = current_idx * chunk_size
            
            # Calculate the product of m, n, o for the current chunk only
            total_combinations = len(m) * len(n) * len(o)
            if start_idx + chunk_size <= total_combinations:
                combinations = list(product(m, n, o))[start_idx:start_idx + chunk_size]
            else:
                combinations = list(product(m, n, o))[start_idx:]
                
            expected_models = set(zip(map(str, [x[0] for x in combinations]), 
                                    map(str, [x[1] for x in combinations]), 
                                    map(str, [x[2] for x in combinations])))
            missing_models = expected_models - current_models
            if missing_models:
                print(f"Missing models in {file}: {missing_models}")
        else:
            print(f"{file} has the expected number of rows: {len(df) - 1}.")
    except Exception as e:
        print(f"Error reading {file}: {e}")


Reading 5 files...
/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/Sim 7_summary_1.csv has the expected number of rows: 1000.
/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/Sim 7_summary_2.csv has the expected number of rows: 1000.
/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/Sim 7_summary_3.csv has the expected number of rows: 1000.
/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/Sim 7_summary_4.csv has the expected number of rows: 1000.
/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/Sim 7_summary_5.csv has the expected number of rows: 1000.


In [3]:
from itertools import product

base_path = '/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/'
sim_version = 'Sim 9'

m = [round(x, 5) for x in np.linspace(0.001, 0.1, 1000)]
n = [round(x, 5) for x in np.linspace(0, 360, 1000)]

chunk_size = 1000

num_files = (len(m) * len(n)) // chunk_size

# Read all files 
files = [f"{base_path}{sim_version}_summary_{i+1}.csv" for i in range(num_files)]
print(f"Reading {len(files)} files...")

model_name = 'POW'
macro_model_params = model_name.strip().split('_')[0]
macro_columns = model_params[macro_model_params]

columns=['strength', 'pa', 'num_images', 'pos_rms', 'mag_rms', 't_mpole_str', 't_mpole_pa', 'chi2'] + macro_columns

# Read individual files and check if all models are present 
for file in files:
    try:
        df = pd.read_csv(file, names=columns)
        if len(df) - 1 != chunk_size:
            print(f"Warning: {file} has {len(df) - 1} rows, expected {chunk_size}.")
            # Create tuples of (strength, pa) from current file
            current_models = set(zip(df['strength'].astype(str), df['pa'].astype(str)))
            # Create expected models for current chunk based on indices
            current_idx = int(file.split('_')[-1].split('.')[0]) - 1
            start_idx = current_idx * chunk_size
            
            # Calculate the product of m, n for the current chunk only
            total_combinations = len(m) * len(n)
            if start_idx + chunk_size <= total_combinations:
                combinations = list(product(m, n))[start_idx:start_idx + chunk_size]
            else:
                combinations = list(product(m, n))[start_idx:]

            expected_models = set(zip(map(str, [x[0] for x in combinations]), 
                                    map(str, [x[1] for x in combinations])))
            missing_models = expected_models - current_models
            if missing_models:
                print(f"Missing models in {file}: {missing_models}")
        else:
            print(f"{file} has the expected number of rows: {len(df) - 1}.")
    except Exception as e:
        print(f"Error reading {file}: {e}")


Reading 1000 files...
/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/Sim 9_summary_1.csv has the expected number of rows: 1000.
/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/Sim 9_summary_2.csv has the expected number of rows: 1000.
/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/Sim 9_summary_3.csv has the expected number of rows: 1000.
/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/Sim 9_summary_4.csv has the expected number of rows: 1000.
/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/Sim 9_summary_5.csv has the expected number of rows: 1000.
/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/Sim 9_summary_6.csv has the expected number of rows: 1000.
/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/Sim 9_summary_7.csv has the expected number of rows: 1000.
/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG L

In [17]:
def rms_extract(model_ver, model_path, constraint):
    global pos_rms, mag_rms, chi2_value
    # Load the data
    with open(model_path + '/' + model_ver + '_optresult' + '.dat', 'r') as file:
        opt_result = file.readlines()

    # Find the last line with 'optimize' in it
    last_optimize_index = None
    for idx in range(len(opt_result) - 1, -1, -1):
        if 'optimize' in opt_result[idx]:
            last_optimize_index = idx
            last_optimize_line = opt_result[idx]
            break
    if last_optimize_index is None:
        raise ValueError("No line with 'optimize' found in the file.")

    # Extract everything after the last 'optimize' line
    opt_result = opt_result[last_optimize_index + 1:]

    # Count the number of lines that start with 'lens'
    lens_count = sum(1 for line in opt_result if line.startswith('lens'))

    # Initialize a dictionary to hold the lens parameters
    lens_params_dict = {}

    # Extract the lens parameters
    lens_params = []
    for line in opt_result:
        if line.startswith('lens'):
            parts = re.split(r'\s+', line.strip())
            lens_name = parts[1]
            params = [float(x) for x in parts[2:]]

            # Store the parameters in the dictionary
            lens_params_dict[lens_name] = params
            lens_params.append((lens_name, params))

    # Remove the first lens parameter
    if lens_params:
        for i in range(len(lens_params)):
            lens_name, params = lens_params[i]
            lens_params_dict[lens_name] = params[1:]
    
    # Extract the chi2 
    chi2_line = next((line for line in opt_result if 'chi^2' in line), None)
    if chi2_line is None:
        raise ValueError("No line with 'chi2' found in the file.")

    chi2_value = float(chi2_line.split('=')[-1].strip().split()[0])
    print(f"✅ Extracted chi2 value: {chi2_value}")

    # Number of len profiles
    num_lens_profiles = len(lens_params_dict)

    # Use generic column names: param1, param2, ...
    df = pd.DataFrame()
    rows = []
    max_param_len = 0

    for lens_name, params in lens_params_dict.items():
        row = {'Lens Name': lens_name}
        for i, val in enumerate(params):
            row[f'param{i+1}'] = val
        rows.append(row)
        if len(params) > max_param_len:
            max_param_len = len(params)

    columns = ['Lens Name'] + [f'param{i+1}' for i in range(max_param_len)]
    df = pd.DataFrame(rows, columns=columns)
    
    # Load the input parameters from the Python file
    with open('/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/POS+MAG/SIE+SHEAR/pos_point.py', 'r') as file:
        py = file.readlines()

    # Extracting the input parameters from the Python file
    set_lens_lines = [line for line in py if line.startswith('glafic.set_lens(')]
    if not set_lens_lines:
        raise ValueError("No lines starting with 'glafic.set_lens(' found in the file.")

    set_lens_params = []
    for line in set_lens_lines:
        match = re.search(r'set_lens\((.*?)\)', line)
        if match:
            params_str = match.group(1)
            params = [param.strip() for param in params_str.split(',')]
            set_lens_params.append(params)
        else:
            raise ValueError(f"No valid parameters found in line: {line.strip()}")

    # Store the parameters in a dictionary
    set_lens_dict = {}
    for params in set_lens_params:
        if len(params) < 3:
            raise ValueError(f"Not enough parameters found in line: {params}")
        lens_name = params[1].strip("'\"")  # Remove quotes from lens name
        lens_params = [float(x) for x in params[2:]]  # Skip index and lens name
        set_lens_dict[lens_name] = lens_params

    # Remove the first lens parameter
    if set_lens_dict:
        for lens_name, params in set_lens_dict.items():
            set_lens_dict[lens_name] = params[1:]  # Remove the first parameter (index)

    # Use generic column names: param1, param2, ...
    df_input = pd.DataFrame()
    rows_input = []
    max_param_len_input = 0
    for lens_name, params in set_lens_dict.items():
        row = {'Lens Name': lens_name}
        for i, val in enumerate(params):
            row[f'param{i+1}'] = val
        rows_input.append(row)
        if len(params) > max_param_len_input:
            max_param_len_input = len(params)
    columns_input = ['Lens Name'] + [f'param{i+1}' for i in range(max_param_len_input)]
    df_input = pd.DataFrame(rows_input, columns=columns_input)
    
    # Extract input flags from the Python file
    set_flag_lines = [line for line in py if line.startswith('glafic.setopt_lens(')]
    if not set_flag_lines:
        raise ValueError("No lines starting with 'glafic.setopt_lens(' found in the file.")
    set_flag_params = []
    for line in set_flag_lines:
        match = re.search(r'setopt_lens\((.*?)\)', line)
        if match:
            params_str = match.group(1)
            params = [param.strip() for param in params_str.split(',')]
            set_flag_params.append(params)
        else:
            raise ValueError(f"No valid parameters found in line: {line.strip()}")
    
    # Store the parameters in a dictionary
    set_flag_dict = {}
    for params in set_flag_params:
        if len(params) < 2:
            raise ValueError(f"Not enough parameters found in line: {params}")
        # The lens name is not present in setopt_lens, so use the lens index to map to set_lens_dict
        lens_index = params[0].strip("'\"")
        # Find the lens name corresponding to this index from set_lens_params
        lens_name = None
        for lens_params in set_lens_params:
            if lens_params[0].strip("'\"") == lens_index:
                lens_name = lens_params[1].strip("'\"")
                break
        if lens_name is None:
            raise ValueError(f"Lens name for index {lens_index} not found in set_lens_params")
        flag = ','.join(params[1:])  # Join all flag values as a string
        set_flag_dict[lens_name] = flag
   
    # Remove the first flag parameter
    if set_flag_dict:
        for lens_name, flag in set_flag_dict.items():
            flag_parts = flag.split(',')
            set_flag_dict[lens_name] = ','.join(flag_parts[1:])  # Remove the first flag parameter
    
    # Dynamically create columns: 'Lens Name', 'flag1', 'flag2', ..., based on the maximum number of flags
    df_flag = pd.DataFrame()
    rows_flag = []
    max_flag_len = 0
    
    # First, determine the maximum number of flags
    for flag in set_flag_dict.values():
        flag_parts = flag.split(',')
        if len(flag_parts) > max_flag_len:
            max_flag_len = len(flag_parts)
    for lens_name, flag in set_flag_dict.items():
        flag_parts = flag.split(',')
        row = {'Lens Name': lens_name}
        for i, val in enumerate(flag_parts):
            row[f'flag{i+1}'] = val
        rows_flag.append(row)
    columns_flag = ['Lens Name'] + [f'flag{i+1}' for i in range(max_flag_len)]  
    df_flag = pd.DataFrame(rows_flag, columns=columns_flag)
    
    # Combine all dataframes into a list of dataframes for each lens
    dfs = []
    
    for i in range(num_lens_profiles):
        lens_name = df['Lens Name'][i]
        
        # Find the model type (case-insensitive match)
        model_type = None
        for m in model_list:
            if m.lower() == lens_name.lower():
                model_type = m
                break
        if model_type is None:
            continue

        symbols = model_params[model_type][:7]
        # Row 2: input
        row_input = pd.DataFrame([df_input.iloc[i, 1:8].values], columns=symbols)
        # Row 3: output
        row_output = pd.DataFrame([df.iloc[i, 1:8].values], columns=symbols)
        # Row 4: flags
        row_flags = pd.DataFrame([df_flag.iloc[i, 1:8].values], columns=symbols)

        # Stack vertically, add a label column for row type
        lens_df = pd.concat([
            row_input.assign(Type='Input'),
            row_output.assign(Type='Output'),
            row_flags.assign(Type='Flag')
        ], ignore_index=True)
        lens_df.insert(0, 'Lens Name', lens_name)
        
        # Move 'Type' to the second column
        cols = lens_df.columns.tolist()
        cols.insert(1, cols.pop(cols.index('Type')))
        lens_df = lens_df[cols]
        dfs.append(lens_df)
    
    # Anomaly Calculation
    columnn_names = ['x', 'y', 'mag', 'pos_err', 'mag_err', '1', '2', '3']
    obs_point = pd.read_csv('/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/obs_point/obs_point_(POS+FLUX).dat', delim_whitespace=True, header=None, skiprows=1, names=columnn_names)
    out_point = pd.read_csv(model_path + '/' + model_ver + '_point.dat', delim_whitespace=True, header=None, skiprows=1, names=columnn_names)
    out_point.drop(columns=['mag_err', '1', '2', '3'], inplace=True)

    # Drop rows in obs_point where the corresponding out_point['mag'] < 1
    mask = abs(out_point['mag']) >= 1
    out_point = out_point[mask[:len(out_point)]].reset_index(drop=True)
    out_point['x_diff'] = abs(out_point['x'] - obs_point['x'])
    out_point['y_diff'] = abs(out_point['y'] - obs_point['y'])
    out_point['mag_diff'] = abs(abs(out_point['mag']) - abs(obs_point['mag']))
    out_point['pos_sq'] = np.sqrt((out_point['x_diff']**2 + out_point['y_diff']**2).astype(float))  # Plotted on graph

    # RMS
    pos_rms = np.average(out_point['pos_sq'])

    mag_rms = np.average(np.sqrt((out_point['mag_diff']**2).astype(float)))

    return pos_rms, mag_rms, dfs, chi2_value

In [18]:
model_output_dir = base_path

for strength, pa, kappa in missing_models:
    model_name = f'SIE_POS_SHEAR_{strength}_{pa}_{kappa}'
    model_path = os.path.join(model_output_dir, model_name)

    print(f"\nProcessing missing model: {model_name}")

    # --- Model Generation ---
    glafic.init(0.3, 0.7, -1.0, 0.7, '/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/left', 20.0, 20.0, 21.56, 21.56, 0.01, 0.01, 1, verb=0)
    glafic.set_secondary('chi2_splane 1', verb=0)
    glafic.set_secondary('chi2_checknimg 1', verb=0)
    glafic.set_secondary('chi2_restart   -1', verb=0)
    glafic.set_secondary('chi2_usemag    1', verb=0)
    glafic.set_secondary('hvary          0', verb=0)
    glafic.set_secondary('ran_seed -122000', verb=0)
    glafic.startup_setnum(2, 0, 1)
    glafic.set_lens(1, 'sie', 0.261343256161012, 1.549839e+02, 20.78, 20.78, 0.107, 23.38, 0.0, 0.0)
    glafic.set_lens(2, 'pert', 0.261343256161012, 1.0, 20.78, 20.78, float(strength), float(pa), 0.0, float(kappa))
    glafic.set_point(1, 1.0, 20.78, 20.78)
    glafic.setopt_lens(1, 0, 1, 1, 1, 1, 1, 0, 0)
    glafic.setopt_lens(2, 0, 0, 0, 0, 0, 0, 0, 0)
    glafic.setopt_point(1, 0, 1, 1)
    glafic.model_init(verb=0)
    glafic.readobs_point('/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/obs_point/obs_point_(POS).dat')
    glafic.optimize()
    glafic.findimg()
    glafic.writecrit(1.0)
    glafic.writelens(1.0)
    glafic.quit()

    columns = ['x', 'y', 'm', 'm_err']
    file_name = 'left_point.dat'

    if os.path.exists(file_name):
        data = pd.read_csv(file_name, delim_whitespace=True, skiprows=1, header=None, names=columns)
        num_images = len(data)
        
        constraint = 'pos'  # Since model name contains 'POS'
        pos_rms, mag_rms, dfs, chi2 = rms_extract('left', '/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test', constraint)

        # Create the result dataframe
        result_df = pd.DataFrame({
            'strength': [strength],
            'pa': [pa],
            'kappa': [kappa],
            'num_images': [num_images],
            'pos_rms': [pos_rms],
            'mag_rms': [mag_rms],
            't_shear_str': [dfs[1]['$\gamma$'][1]],
            't_shear_pa': [dfs[1]['$θ_{\gamma}$'][1]],
            't_shear_kappa': [dfs[1]['$\kappa$'][1]],
            'sie_vel_disp': [dfs[0]['$\sigma$'][1]],
            'sie_pa': [dfs[0]['$θ_{e}$'][1]],
            'sie_ell': [dfs[0]['e'][1]],
            'chi2': [chi2]
        })

        # Find the appropriate chunk file based on the model parameters
        param_idx = list(product(m, n, o)).index((float(strength), float(pa), float(kappa)))
        chunk_num = param_idx // chunk_size + 1
        chunk_file = f'{base_path}{sim_version}_summary_{chunk_num}.csv'
        
        # Read existing data
        existing_df = pd.read_csv(chunk_file)
        
        # Append new data
        updated_df = pd.concat([existing_df, result_df], ignore_index=True)
        
        # Save back to CSV
        updated_df.to_csv(chunk_file, index=False)
        
        print(f"Results appended to {chunk_file}")
        
        # Clean up generated files
        crit_file = '/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/left_crit.dat'
        lens_file = '/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/left_lens.fits'
        point_file = '/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/left_point.dat'
        opt_file = '/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/left_optresult.dat'

        for file_to_delete in [crit_file, lens_file, point_file, opt_file]:
            if os.path.exists(file_to_delete):
                os.remove(file_to_delete)
    else:
        print(f"File {file_name} does not exist.")



Processing missing model: SIE_POS_SHEAR_0.45545_0.0_-0.5
amoeba_delta = 1.000000e+00

amoeba_delta = 4.203005e-01

amoeba_delta = 4.179993e-01

amoeba_delta = 3.715393e-02

n_img = 4
x =   20.7541   y =   20.3041   mag =    1.1211 [  -0.124]   td[day] =     0.000
x =   21.2769   y =   20.9837   mag =   -0.7926 [   0.252]   td[day] =    10.415
x =   20.3961   y =   21.0403   mag =   -1.2640 [  -0.254]   td[day] =    10.121
x =   20.8198   y =   21.2588   mag =   10.4450 [  -2.547]   td[day] =     9.801
✅ Extracted chi2 value: 1323.216


######## optimizing lens
 number of parameters = 5 (lens) + 0 (extend) + 2 (point) 

amoeba:  n =     1  y = 1.901542e+05  tol = 1.512366e+00
amoeba:  n =     2  y = 1.901542e+05  tol = 1.298337e+00
amoeba:  n =     3  y = 1.901542e+05  tol = 1.267455e+00
amoeba:  n =     4  y = 1.901542e+05  tol = 1.126867e+00
amoeba:  n =     5  y = 1.901542e+05  tol = 8.176553e-01
amoeba:  n =     6  y = 1.901542e+05  tol = 7.650674e-01
amoeba:  n =     7  y = 1.901542e+05  tol = 6.225875e-01
amoeba:  n =     8  y = 1.901542e+05  tol = 4.954196e-01
amoeba:  n =     9  y = 1.901542e+05  tol = 4.222131e-01
amoeba:  n =    10  y = 1.901542e+05  tol = 2.956722e-01
amoeba:  n =    11  y = 1.901542e+05  tol = 2.440244e-01
amoeba:  n =    12  y = 1.640347e+05  tol = 3.650932e-01
amoeba:  n =    13  y = 1.640347e+05  tol = 3.339131e-01
amoeba:  n =    14  y = 1.479880e+05  tol = 4.234415e-01
amoeba:  n =    15  y = 1.423541e+05  tol = 3.620442e-01
amoeba:  n =    16  y = 1.423541e+05  tol = 2.875119e-01
amo

Results appended to /Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Test/Sim 6_summary_91.csv


In [4]:
# Concat all dataframes together
df = pd.concat((pd.read_csv(file, names=columns, skiprows=1) for file in files), ignore_index=True)

df.to_csv(f'{base_path}{sim_version}_summary.csv', index=False)